# Data Pre-Processing

### Define Functions

In [1]:
# load setup.py file
%run setup.py
# change number of lines in dataframe
pd.set_option('display.max_rows', 10)

In [2]:
def read_csv_files_from_folder(filepath,folder):
    # Step 1: Create a path to all the .csv files in the folder
    csv_path = os.path.join(filepath, folder, "*.csv")

    # Step 2: Use the glob module to get a list of all the .csv files in the folder
    csv_files = glob.glob(csv_path)
    
    # Step 3: Create an empty list to store the dataframes
    df_list = []

    # Step 4: Loop through all the .csv files and append them to the dataframe list
    for file_path in csv_files:
        df = pd.read_csv(file_path,index_col=False,encoding='utf-8',sep=",")
        df['Conference (short)'] = os.path.basename(file_path)
        df['Conference (short)'] = df['Conference (short)'].str.split('_').str[0]
        df_list.append(df)

    # Step 5: Concatenate all the dataframes in the list into a single dataframe
    final_df = pd.concat(df_list, ignore_index=True)
    
    # Step 6: Delete unnesseary columns
    try:
        final_df = final_df.drop(["Field"], axis=1)
        final_df = final_df.drop(["Key Note Speaker"], axis=1)
    except:
        pass

    # Step 7: Return the final dataframe
    return final_df

def Name_preprocessing(df):
    # First Name
    df['First name'] = df['Full name'].str.split(' ').str[0].str.lower()
    df['First name'] = df["First name"].apply(unidecode)
    
    print("Number of rows where the first name only contains abbreviation: ", df[df['First name'].str.contains('^[a-z]\.$')].shape[0])
    print("These rows will be removed")
    df = df[~df['First name'].str.contains('^[a-z]\.$')]
    print("There are now:",df.shape[0],"rows")
       
    # Middel and Last Name
    df['Middle name'] = df['Full name'].str.split(' ').str[1]
    df['Last name (temp)'] = df['Full name'].str.split(' ').str[2]
    df['Last name'] = np.where(df['Last name (temp)'].isnull(), df['Middle name'], df['Last name (temp)'])
    df['Middle name'] = np.where(df['Last name (temp)'].isnull(), np.nan, df['Middle name'])
    df = df.drop(["Last name (temp)"], axis=1)
    return df

def Middle_name_clean(df):
    # Find all rows with different middle names
    row_to_remove = df[df.duplicated(['First name','Last name'], keep=False)].sort_values(['First name','Last name']).drop_duplicates(subset=['First name','Middle name','Last name'], keep=False).drop_duplicates(subset=['First name','Last name'], keep=False)
    rows_to_change = df[df.duplicated(['First name','Last name'], keep=False)].sort_values(['First name','Last name']).drop_duplicates(subset=['First name','Middle name','Last name'], keep=False)
    rows_to_change = rows_to_change[~rows_to_change.isin(row_to_remove)].dropna(how='all').sort_values(['First name','Last name','Middle name'])

    for i in range(len(rows_to_change)-1):
        if (rows_to_change.iloc[i]['First name'] == rows_to_change.iloc[i+1]['First name']) and (rows_to_change.iloc[i]['Last name'] == rows_to_change.iloc[i+1]['Last name']):
            if pd.isnull(rows_to_change.iloc[i+1]['Middle name']):
                rows_to_change.iloc[i+1,rows_to_change.columns.get_loc('Middle name')] = rows_to_change.iloc[i]['Middle name']
    
    # Overwrite the Middle name
    df.loc[rows_to_change.index,'Middle name'] = rows_to_change['Middle name']
    
    df['First name'] = df['First name'].str.capitalize()

    # if middle name is nan then combine first and last name with space between, otherwise if middle name is not nan then combine first, middle and last name with space between
    df['Full name'] = np.where(df['Middle name'].isnull(), df['First name'] + ' ' + df['Last name'], df['First name'] + ' ' + df['Middle name'] + ' ' + df['Last name'])
    return df



#############################################
# Gender Data
#############################################
name_df = pd.read_csv('Dimension Tables/wgnd_2_0_name-gender-code.csv')
# Keep only US or DK names
name_df = name_df[(name_df['code'] == 'US') | (name_df['code'] == 'DK')]
# drop duplicates
name_df = name_df.drop_duplicates(subset=['name',"gender"], keep='first')
# for all duplicated value keep the row with the max "wgt" value
name_df = name_df.sort_values('wgt', ascending=False).drop_duplicates(subset=['name'], keep='first')

print("Number of rows:", len(name_df))
name_df.head()

Number of rows: 97795


,name,code,gender,wgt
758,aaban,US,M,1.0
1857914,lexxy,US,F,1.0
1857667,lexia,US,F,1.0
1857563,lexi,US,F,1.0
1857549,lexey,US,F,1.0


## Key Note Speakers

#### Load and combine all .csv files in the folder

In [112]:
df = read_csv_files_from_folder(filepath=filepath,folder='Invited Speakers')
print("Number of rows:", len(df))
df.head()

Number of rows: 1856


,Full name,Year,Sex,Conference (short)
0,Atsushi Asada,1995,0,ASPDAC
1,Jim Meadlock,1995,0,ASPDAC
2,John Darringer,1995,0,ASPDAC
3,Tatsuo Izawa,1997,0,ASPDAC
4,Daniel D. Gajski,1997,0,ASPDAC


#### Remove rows

In [113]:
# Only include data after 2003
df = df[(df['Year'] >= 2003) & (df['Year'] <= 2022)]

#### Clean the name columns

In [114]:
df = Name_preprocessing(df)
df.head()

Number of rows where the first name only contains abbreviation:  1
These rows will be removed
There are now: 1343 rows


,Full name,Year,Sex,Conference (short),First name,Middle name,Last name
22,Glovanni De WIichcli,2003,0,ASPDAC,glovanni,De,WIichcli
23,Tadahiro Ohm,2003,0,ASPDAC,tadahiro,NaN,Ohm
24,Ycrvant Zorian,2003,0,ASPDAC,ycrvant,NaN,Zorian
25,Gary L. Baldwi,2004,0,ASPDAC,gary,L.,Baldwi
26,Rudy Lauwereins,2004,0,ASPDAC,rudy,NaN,Lauwereins


#### Add gender data

In [115]:
# merge df and name_df by "First name" and "name" and include "Gender column"
df = pd.merge(df, name_df, how='left', left_on='First name', right_on='name')

# Drop unnecessary columns
df = df.drop(['name', 'code', 'wgt'], axis=1)

print("Number of rows with null value in gender:", df["gender"].isnull().sum())

# Replace with the value in 'Sex' column
df['gender'] = df['gender'].fillna(df['Sex'])

df.head()

Number of rows with null value in gender: 110


,Full name,Year,Sex,Conference (short),First name,Middle name,Last name,gender
0,Glovanni De WIichcli,2003,0,ASPDAC,glovanni,De,WIichcli,0
1,Tadahiro Ohm,2003,0,ASPDAC,tadahiro,NaN,Ohm,0
2,Ycrvant Zorian,2003,0,ASPDAC,ycrvant,NaN,Zorian,0
3,Gary L. Baldwi,2004,0,ASPDAC,gary,L.,Baldwi,M
4,Rudy Lauwereins,2004,0,ASPDAC,rudy,NaN,Lauwereins,M


##### Clean up gender columns

In [119]:
df['Sex'].replace(np.nan, 'unknown', inplace=True)
df['Sex'].replace('x','unknown',inplace=True)
df['Sex'].replace('0','M',inplace=True)
df['Sex'].replace(0,'M',inplace=True)
df['Sex'].replace('1','F',inplace=True)
df['Sex'].replace(1,'F',inplace=True)

df['gender'].replace(1,'F',inplace=True)
df['gender'].replace(0,'M',inplace=True)
df['gender'].replace('1','F',inplace=True)
df['gender'].replace('0','M',inplace=True)

In [121]:
print('Number of rows where "Sex" is F and "gender" is M:',len(np.where((df['Sex'] == 'F') & (df['gender']=='M'))[0]))
print('Number of rows where "Sex" is M and "gender" is F:',len(np.where((df['Sex'] == 'M') & (df['gender']=='F'))[0]))

# Overwrite the value in column 'gender' with the 'Sex' column
df.loc[df['Sex'] == 'F', 'gender'] = 'F'  
df.loc[df['Sex'] == 'M', 'gender'] = 'M'   

# Drop unnecessary columns
df = df.drop(['Sex'],axis=1)

Number of rows where "Sex" is F and "gender" is M: 4
Number of rows where "Sex" is M and "gender" is F: 8


#### Fix unique names

In [122]:
print("Before: There are ",df['Full name'].nunique()," unique names")
df = Middle_name_clean(df)
print("After: There are ",df['Full name'].nunique()," unique names")

Before: There are  1147  unique names
After: There are  1134  unique names


#### Save factTable

In [124]:
df.to_csv(os.path.join(filepath,"factInvited_init.csv"), index=False)

## Proceedings

#### Load and combine all .csv files in the folder

In [15]:
# Append all files in Data folder as a datafrom, and add a column for the file name without the extension and folder name
df = read_csv_files_from_folder(filepath=filepath,folder='Proceedings')
print("Number of rows:", len(df))
df = df.drop(['Full_name'], axis=1)

Number of rows: 283579


In [17]:
df.head()

,Full name,Year,Links,Conference (short)
0,Hiroto Yasuura,2003,['https://dblp.org/pid/43/4149.html'],ASPDAC
1,Yazdan Aghaghiri,2003,['https://dblp.org/pid/20/4429.html'],ASPDAC
2,Farzan Fallah,2003,['https://dblp.org/pid/02/1886.html'],ASPDAC
3,Massoud Pedram,2003,['https://dblp.org/pid/p/MassoudPedram.html'],ASPDAC
4,Satoshi Komatsu,2003,['https://dblp.org/pid/08/4585.html'],ASPDAC


#### Remove rows

In [19]:
# Only include data after 2003
df['Year'] = df['Year'].astype(int)
df = df[(df['Year'] >= 2003) & (df['Year'] <= 2022)]
len(df)

283579

#### Clean the names

In [20]:
df = Name_preprocessing(df)
df.head()

Number of rows where the first name only contains abbreviation:  1836
These rows will be removed
There are now: 281743 rows


,Full name,Year,Links,Conference (short),First name,Middle name,Last name
0,Hiroto Yasuura,2003,['https://dblp.org/pid/43/4149.html'],ASPDAC,hiroto,NaN,Yasuura
1,Yazdan Aghaghiri,2003,['https://dblp.org/pid/20/4429.html'],ASPDAC,yazdan,NaN,Aghaghiri
2,Farzan Fallah,2003,['https://dblp.org/pid/02/1886.html'],ASPDAC,farzan,NaN,Fallah
3,Massoud Pedram,2003,['https://dblp.org/pid/p/MassoudPedram.html'],ASPDAC,massoud,NaN,Pedram
4,Satoshi Komatsu,2003,['https://dblp.org/pid/08/4585.html'],ASPDAC,satoshi,NaN,Komatsu


#### Add gender data

In [21]:
# merge df and name_df by "First name" and "name" and include "Gender column"
df = pd.merge(df, name_df, how='left', left_on='First name', right_on='name')

# Drop unnecessary columns
df = df.drop(['name', 'code', 'wgt'], axis=1)

print("Number of rows with null value in gender:", df["gender"].isnull().sum())

# Drop all rows with nan value in gender column
df = df.dropna(subset=['gender'])
print("Number of rows after removal of unknown genders:", len(df))

df.head()

Number of rows with null value in gender: 79663
Number of rows after removal of unknown genders: 202080


,Full name,Year,Links,Conference (short),First name,Middle name,Last name,gender
0,Hiroto Yasuura,2003,['https://dblp.org/pid/43/4149.html'],ASPDAC,hiroto,NaN,Yasuura,M
2,Farzan Fallah,2003,['https://dblp.org/pid/02/1886.html'],ASPDAC,farzan,NaN,Fallah,M
4,Satoshi Komatsu,2003,['https://dblp.org/pid/08/4585.html'],ASPDAC,satoshi,NaN,Komatsu,M
5,Masahiro Fujita,2003,['https://dblp.org/pid/56/1768.html'],ASPDAC,masahiro,NaN,Fujita,M
6,Sri Parameswaran,2003,['https://dblp.org/pid/38/622.html'],ASPDAC,sri,NaN,Parameswaran,F


#### Fix Unique names

In [22]:
print("Before: There are ",df['Full name'].nunique()," unique names")
df = Middle_name_clean(df)
print("After: There are ",df['Full name'].nunique()," unique names")

Before: There are  69321  unique names
After: There are  68772  unique names


#### Save factTable

In [23]:
df.head()

,Full name,Year,Links,Conference (short),First name,Middle name,Last name,gender
0,Hiroto Yasuura,2003,['https://dblp.org/pid/43/4149.html'],ASPDAC,Hiroto,NaN,Yasuura,M
2,Farzan Fallah,2003,['https://dblp.org/pid/02/1886.html'],ASPDAC,Farzan,NaN,Fallah,M
4,Satoshi Komatsu,2003,['https://dblp.org/pid/08/4585.html'],ASPDAC,Satoshi,NaN,Komatsu,M
5,Masahiro Fujita,2003,['https://dblp.org/pid/56/1768.html'],ASPDAC,Masahiro,NaN,Fujita,M
6,Sri Parameswaran,2003,['https://dblp.org/pid/38/622.html'],ASPDAC,Sri,NaN,Parameswaran,F


In [28]:
df.to_csv(os.path.join(filepath,"factProceedings_init.csv"), index=False)

## Add Columns
NB: Do not complete this step until after the Get_Paper_info.ipynb has been runned

In [2]:
# Load files
df_Invited = pd.read_csv(os.path.join(filepath,'factInvited - BU29042023.csv'))
# df_Proceedings = pd.read_csv(os.path.join(filepath,'factProceedings.csv'))

In [7]:
df_Invited.loc[711,'gender'] = "M"

In [8]:
df_Invited[df_Invited['First name'] == "Dana"]

,Full name,Year,Conference (short),First name,Middle name,Last name,gender,Paper Count,Max Paper Count,First year paper,Age,Conference (long),Main Topic,Productivity
13,Dana S. Nau,2005,AAAI,Dana,S.,Nau,F,111,186,1980,25,AAAI Conference on Artificial Intelligence,Artificial Intelligence,4.440000
711,Dana S. Scott,2006,LICS,Dana,S.,Scott,M,35,48,1958,48,Symposium on Logic in Computer Science,Algorithm and Theory,0.729167
889,Dana Pe'er,2019,NeurIPS,Dana,NaN,Pe'er,F,16,16,1999,20,Conference on Neural Information Processing Sy...,Artificial Intelligence,0.800000
1062,Dana Randall,2010,SWAT,Dana,NaN,Randall,F,40,77,1987,23,Scandinavian Symposium and Workshops on Algori...,Algorithm and Theory,1.739130
1278,Dana Bartosova,2016,WoLLIC,Dana,NaN,Bartosova,F,0,2,2017,0,"Workshop on Logic, Language, Information and C...",Artificial Intelligence,0.000000


In [5]:
df_Proceedings[df_Proceedings['Main Topic'].isna()]['Conference (short)'].unique()

array(['HiPC', 'SC', 'WoLLIC'], dtype=object)

In [7]:
df_Proceedings[df_Proceedings['Conference (short)'] == "Concurrent, distributed and parallel computing"]

,Full name,Year,Conference (short),First name,Middle name,Last name,gender,column_equal,Paper Count,Max Paper Count,First year paper,ID,Age,Conference (long),Main Topic,Productivity


In [6]:
df_Proceedings[df_Proceedings['Conference (short)'].isin("HiPC"]

,Full name,Year,Conference (short),First name,Middle name,Last name,gender,column_equal,Paper Count,Max Paper Count,First year paper,ID,Age,Conference (long),Main Topic,Productivity
66189,Abhishek P. Patil,2003,HiPC,Abhishek,P.,Patil,M,True,2,10,2003,Abhishek P. Patil_1,0,NaN,NaN,0.000000
66190,Adriano Veloso,2003,HiPC,Adriano,NaN,Veloso,M,True,9,124,2001,Adriano Veloso_1,2,NaN,NaN,4.500000
66191,Alessandro Mei,2003,HiPC,Alessandro,NaN,Mei,M,True,16,90,1997,Alessandro Mei_1,6,NaN,NaN,2.666667
66192,Antonio González,2003,HiPC,Antonio,NaN,González,M,False,116,310,1988,Antonio González_1,15,NaN,NaN,7.733333
66193,Arnaud Contes,2003,HiPC,Arnaud,NaN,Contes,M,True,1,6,2003,Arnaud Contes_1,0,NaN,NaN,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68358,Xiaojing An,2021,HiPC,Xiaojing,NaN,An,F,True,7,9,2015,Xiaojing An_1,6,NaN,NaN,1.166667
68359,Xing Cai,2021,HiPC,Xing,NaN,Cai,F,True,73,78,1997,Xing Cai_1,24,NaN,NaN,3.041667
68360,Xu T. Liu,2021,HiPC,Xu,T.,Liu,M,True,4,9,2017,Xu T. Liu_1,4,NaN,NaN,1.000000
68361,Yuan Meng,2021,HiPC,Yuan,NaN,Meng,M,True,50,60,2013,Yuan Meng_1,8,NaN,NaN,6.250000


In [46]:
IC2S2_data = IC2S2_data.sort_values(['Full name', 'Links'])
IC2S2_data['ID'] = 0
current_name = ''
current_links = ''
count = 0

for index, row in IC2S2_data.iterrows():
    if row['Full name'] != current_name:
        current_name = row['Full name']
        current_links = row['Links']
        count = 1
    elif row['Links'] != current_links:
        current_links = row['Links']
        count += 1
    IC2S2_data.at[index, 'ID'] = row['Full name'] + "_" + str(count)

In [47]:
IC2S2_data

,Full name,Year,Conference (short),First name,Middle name,Last name,gender,Paper Count,Max Paper Count,First year paper,Links,ID
69261,Aamena Alshamsi,2019,IC2S2,Aamena,NaN,Alshamsi,F,3,6,2014,NaN,Aamena Alshamsi_1
70601,Aaron Clauset,2021,IC2S2,Aaron,NaN,Clauset,M,27,28,2004,NaN,Aaron Clauset_1
71101,Aaron Clauset,2022,IC2S2,Aaron,NaN,Clauset,M,28,28,2004,NaN,Aaron Clauset_2
69892,Aaron Halfaker,2020,IC2S2,Aaron,NaN,Halfaker,M,34,37,2009,NaN,Aaron Halfaker_1
71102,Aaron Nichols,2022,IC2S2,Aaron,NaN,Nichols,M,1,1,2017,NaN,Aaron Nichols_1
...,...,...,...,...,...,...,...,...,...,...,...,...
70600,Zoltan Kmetty,2020,IC2S2,Zoltan,NaN,Kmetty,M,2,4,2018,NaN,Zoltan Kmetty_2
71100,Zoltan Kmetty,2021,IC2S2,Zoltan,NaN,Kmetty,M,3,4,2018,NaN,Zoltan Kmetty_3
71651,Zoltan Kmetty,2022,IC2S2,Zoltan,NaN,Kmetty,M,4,4,2018,NaN,Zoltan Kmetty_4
69890,Zoya Khan,2019,IC2S2,Zoya,NaN,Khan,F,1,2,2019,NaN,Zoya Khan_1


In [49]:
df_Proceedings = pd.concat([df_Proceedings, IC2S2_data], ignore_index=True)

In [51]:
df_Proceedings = df_Proceedings.drop(['Links'], axis=1)

In [52]:
# Add age
# df_Invited['Age'] = np.where(df_Invited['First year paper'] == 0, 0, np.where(df_Invited['Year'] - df_Invited['First year paper'] < 0, 0, df_Invited['Year'] - df_Invited['First year paper']))
df_Proceedings['Age'] = np.where(df_Proceedings['First year paper'] == 0, 0, np.where(df_Proceedings['Year'] - df_Proceedings['First year paper'] < 0, 0, df_Proceedings['Year'] - df_Proceedings['First year paper']))

In [53]:
# Change column types to int
def change_column_types_to_int(df, columns):
    for column in columns:
        df[column] = df[column].astype(int)
    return df

# df_Invited = change_column_types_to_int(df_Invited,['Year','Paper Count','Max Paper Count','First year paper','Age'])
df_Proceedings = change_column_types_to_int(df_Proceedings,['Year','Paper Count','Max Paper Count','First year paper','Age'])

In [30]:
df_Proceedings.head()

,Full name,Year,Conference (short),First name,Middle name,Last name,gender,column_equal,Paper Count,Max Paper Count,First year paper,ID,Age,Productivity,Conference (long),Main Topic
0,Aaron Gage,2004,AAAI,Aaron,NaN,Gage,M,True,7,8,1999,Aaron Gage_1,5,1.400000,AAAI Conference on Artificial Intelligence,Artificial Intelligence
1,Abhi Shelat,2004,AAAI,Abhi,NaN,Shelat,M,True,7,75,2002,Abhi Shelat_1,2,3.500000,AAAI Conference on Artificial Intelligence,Artificial Intelligence
2,Adele E. Howe,2004,AAAI,Adele,E.,Howe,F,True,48,91,1986,Adele E. Howe_1,18,2.666667,AAAI Conference on Artificial Intelligence,Artificial Intelligence
3,Akihiro Kishimoto,2004,AAAI,Akihiro,NaN,Kishimoto,M,True,4,62,2002,Akihiro Kishimoto_1,2,2.000000,AAAI Conference on Artificial Intelligence,Artificial Intelligence
4,Al Globus,2004,AAAI,Al,NaN,Globus,M,True,6,6,1990,Al Globus_1,14,0.428571,AAAI Conference on Artificial Intelligence,Artificial Intelligence


In [28]:
df_Proceedings = df_Proceedings.drop(['Conference (long)','Main Topic'], axis=1)

In [29]:
# Add field
# df_conference = pd.read_csv(os.path.join(filepath,'Dimension Tables','Conference_Field_Name.csv'),sep=",")
df_conference = pd.read_csv(os.path.join(filepath,'Dimension Tables','dimConference.csv'),sep=",")

# df_Invited = df_Invited.merge(df_conference, how='left', on='Conference (short)')
df_Proceedings = df_Proceedings.merge(df_conference, how='left', on='Conference (short)')

# df_Invited = df_Invited.drop(['Subcategori Topic'],axis=1)
df_Proceedings = df_Proceedings.drop(['Subcategori Topic'],axis=1)

In [25]:
df_conference.iloc[53]

Conference (long)                          Supercomputing Conference
Conference (short)                                                SC
Main Topic            Concurrent, distributed and parallel computing
Subcategori Topic                         High-Performance Computing
Name: 53, dtype: object

In [55]:
# Add Productivity column
# df_Invited['Productivity'] = np.divide(df_Invited['Paper Count'],df_Invited['Age'])
# df_Invited['Productivity'] = df_Invited['Productivity'].replace([np.inf, -np.inf,np.nan], 0)

df_Proceedings['Productivity'] = np.divide(df_Proceedings['Paper Count'],df_Proceedings['Age'])
df_Proceedings['Productivity'] = df_Proceedings['Productivity'].replace([np.inf, -np.inf,np.nan], 0)

In [60]:
df_Proceedings.sort_values(['Conference (short)', 'Year','Full name'], inplace=True).reset_index()

AttributeError: 'NoneType' object has no attribute 'reset_index'

In [130]:
df_Invited['Conference (short)'].replace("ic2s2", "IC2S2", inplace=True)

In [131]:
# Remove because I do not have proceedings from before 2019
df1 = df_Invited[(df_Invited['Conference (short)'] == "IC2S2") & (df_Invited['Year'] < 2019)]
df_Invited = df_Invited[~df_Invited.index.isin(df1.index)]

In [62]:
df_Proceedings.reset_index()

,index,Full name,Year,Conference (short),First name,Middle name,Last name,gender,column_equal,Paper Count,Max Paper Count,First year paper,ID,Age,Conference (long),Main Topic,Productivity
0,179,Aaron Gage,2004,AAAI,Aaron,NaN,Gage,M,True,7,8,1999,Aaron Gage_1,5,AAAI Conference on Artificial Intelligence,Artificial Intelligence,1.400000
1,684,Abhi Shelat,2004,AAAI,Abhi,NaN,Shelat,M,True,7,75,2002,Abhi Shelat_1,2,AAAI Conference on Artificial Intelligence,Artificial Intelligence,3.500000
2,1704,Adele E. Howe,2004,AAAI,Adele,E.,Howe,F,True,48,91,1986,Adele E. Howe_1,18,AAAI Conference on Artificial Intelligence,Artificial Intelligence,2.666667
3,3053,Akihiro Kishimoto,2004,AAAI,Akihiro,NaN,Kishimoto,M,True,4,62,2002,Akihiro Kishimoto_1,2,AAAI Conference on Artificial Intelligence,Artificial Intelligence,2.000000
4,3366,Al Globus,2004,AAAI,Al,NaN,Globus,M,True,6,6,1990,Al Globus_1,14,AAAI Conference on Artificial Intelligence,Artificial Intelligence,0.428571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194633,167597,Tiziano Dalmonte,2022,WoLLIC,Tiziano,NaN,Dalmonte,M,True,15,15,2018,Tiziano Dalmonte_1,4,NaN,NaN,3.750000
194634,168565,Tomas Svoboda,2022,WoLLIC,Tomas,NaN,Svoboda,M,True,47,49,1997,Tomas Svoboda_1,25,NaN,NaN,1.880000
194635,168940,Tomoyuki Yamakami,2022,WoLLIC,Tomoyuki,NaN,Yamakami,M,True,98,98,1992,Tomoyuki Yamakami_1,30,NaN,NaN,3.266667
194636,171264,Valentin Cassano,2022,WoLLIC,Valentin,NaN,Cassano,M,True,13,13,2013,Valentin Cassano_1,9,NaN,NaN,1.444444


In [9]:
# Save tables
df_Invited.to_csv(os.path.join(filepath,'factInvited.csv'), index=False)
# df_Proceedings.to_csv(os.path.join(filepath,'factProceedings.csv'), index=False)